# PERP-UNI NVP Spread Investment Monitor

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import data
import analytics

%load_ext autoreload
%autoreload 2

## Inputs

In [2]:
start_date = '2020-09-09'
end_date = '2021-09-24'
dexs = [
    'uniswap_v1',
    'uniswap_v2',
    'uniswap',
    'perpetual_protocol',
]
coins = [
    'uniswap',
    'perpetual-protocol',
]
benchmark = 'defipulse-index'
nvf_avg_window = 30
annualization_factor = 365
uni_trade_fee = 0.003
perp_trade_fee = 0.001
initial_capital = 1978.28
risk_free_rate = 0

## Get Data

In [3]:
# Call CoinGecko for market data
coins.append(benchmark)
dex_trading_volume = data.dex_trading_volume(dexs, start_date, end_date)
coin_market_data = data.coin_market_data(coins, start_date, end_date)

# Prepare DEX trading volume data
dex_trading_volume['uniswap'].fillna(0, inplace=True)
dex_trading_volume['uniswap_v2'].fillna(0, inplace=True)
dex_trading_volume['uniswap_v1'].fillna(0, inplace=True)
uni_tv = dex_trading_volume['uniswap'] + dex_trading_volume['uniswap_v2'] + dex_trading_volume['uniswap_v1']
uni_tv = uni_tv[uni_tv != 0]
perp_tv = dex_trading_volume['perpetual_protocol'].dropna()

# Prepare coin and benchmark market data
uni_price = coin_market_data['uniswap price'].dropna()
uni_mc = coin_market_data['uniswap market cap'].dropna()
perp_price = coin_market_data['perpetual-protocol price'].dropna()
perp_mc = coin_market_data['perpetual-protocol market cap'].dropna()
benchmark_price = coin_market_data['defipulse-index price'].dropna()
benchmark_mc = coin_market_data['defipulse-index market cap'].dropna()

# Import and format trade data
funding_payments = pd.read_csv('input/funding_payments.csv', index_col='time', parse_dates=True)
pnl = pd.read_csv('input/pnl.csv', index_col='day')
pnl = pnl.drop(' Total')
pnl.index = pnl.index.map(lambda x: pd.to_datetime(float(x), unit='s'))

# Prepare trade data
funding_payments = funding_payments.groupby('future').resample('D').sum()
perp_fp = funding_payments.loc['PERP-PERP']['payment']
perp_fp.index = perp_fp.index.tz_localize(None)
uni_fp = funding_payments.loc['UNI-PERP']['payment']
uni_fp.index = uni_fp.index.tz_localize(None)
perp_pnl = pnl['PERP-PERP'].sort_index()
uni_pnl = pnl['UNI-PERP'].sort_index()

Getting DEX trading volume...
Getting coin market data...
- bitcoin
Got coin market data
- perpetual_protocol
- uniswap
- uniswap_v2
- uniswap_v1
Got DEX trading volume
Getting coin market data...
- perpetual-protocol
- uniswap
- defipulse-index
Got coin market data


## Compute Analytics

In [4]:
# NVF
uni_nvf = analytics.nvf(uni_mc, uni_tv, uni_trade_fee, nvf_avg_window, annualization_factor)
perp_nvf = analytics.nvf(perp_mc, perp_tv, perp_trade_fee, nvf_avg_window, annualization_factor)

# NVF spread (absolute & % diff)
abs_nvf_spread = (perp_nvf - uni_nvf).dropna()
perc_nvf_spread = (abs_nvf_spread / uni_nvf).dropna()

# Determine number of trading days
first_trade_date = min(uni_pnl.index)
today = pd.Timestamp.today()
trade_days = (today - first_trade_date).days

# Overview trade stats
trade_perf = analytics.pnl_performance(perp_pnl+uni_pnl, initial_capital)
total_ret = analytics.total_return(trade_perf)
trade_ann_ret = (annualization_factor / trade_days) * total_ret
trade_log_ret = analytics.returns(trade_perf, 'log')
trade_arith_ret = analytics.returns(trade_perf, 'arith')
benchmark_log_ret = analytics.returns(benchmark_price, 'log')
benchmark_arith_ret = analytics.returns(benchmark_price, 'arith')
trade_ann_vol = np.sqrt(annualization_factor) * analytics.volatility(trade_log_ret)
trade_sr = analytics.sharpe_ratio(trade_log_ret, risk_free_rate, annualization_factor)
trade_ir = analytics.information_ratio(trade_arith_ret, benchmark_arith_ret)
trade_beta = analytics.beta(trade_log_ret, benchmark_log_ret)
trade_alpha = analytics.alpha(trade_log_ret, benchmark_log_ret, risk_free_rate, annualization_factor)
trade_tr = analytics.treynor_ratio(trade_log_ret, benchmark_log_ret, risk_free_rate, annualization_factor)
trade_md = analytics.max_drawdown(trade_arith_ret)

## Present Analytics

In [5]:
# Overview trade stats (all, 30 days, 60 days, etc.)
all_trade_stats = {
    'Total return': total_ret,
    'Annualized return': trade_ann_ret,
    'Volatility': trade_ann_ret,
    'Sharpe ratio': trade_sr,
    'Information ratio': trade_ir,
    'Treynor ratio': trade_tr,
    'Alpha': trade_alpha,
    'Beta': trade_beta,
    'Max drawdown': trade_md,
}
trade_stats = pd.DataFrame(all_trade_stats, index=['All'])
trade_stats

# TODO: Create charts

,Total return,Annualized return,Volatility,Sharpe ratio,Information ratio,Treynor ratio,Alpha,Beta,Max drawdown
All,0.016611,0.866154,0.866154,2.462028,0.389309,6.937949,0.742656,0.123817,-0.04124


## Output Analytics